In [2]:
import pandas as pd
import numpy as np
import gensim
from pymystem3 import Mystem

In [27]:
data = pd.read_csv('dataset.csv')
print(data)

             Lemma_1          Lemma_2                           Type  \
0               вода              сыр  Неоднокоренные (разные корни)   
1               коса             зябь  Неоднокоренные (разные корни)   
2              любой         водитель  Неоднокоренные (разные корни)   
3                нож          красить  Неоднокоренные (разные корни)   
4           красивый       переносица  Неоднокоренные (разные корни)   
..               ...              ...                            ...   
695          восторг        ликование                       Синонимы   
696         циничный  безнравственный                       Синонимы   
697  состоятельность     зажиточность                       Синонимы   
698     оригинальный       самобытный                       Синонимы   
699       лапидарный       лаконичный                       Синонимы   

     isCognate  
0            0  
1            0  
2            0  
3            0  
4            0  
..         ...  
695          0  

# POS-TAG of Data

In [4]:
unique_words = np.unique(data[['Lemma_1', 'Lemma_2']].values)

print(len(unique_words))
unique_words

722


array(['абсолютный', 'актер', 'активный', 'анализ', 'анекдот', 'анкета',
       'аргумент', 'аромат', 'артист', 'банально', 'бегать', 'бегущий',
       'беда', 'бежать', 'безногий', 'безнравственный', 'беседа',
       'бесшумный', 'бланк', 'блестеть', 'блестящий', 'блистание',
       'блистательно', 'блистательный', 'блистать', 'блёстка', 'болезнь',
       'бур', 'бурить', 'бурный', 'буровой', 'бурый', 'буря', 'бурящий',
       'вариант', 'веселье', 'вздернуть', 'вздирать', 'взлет', 'взрослый',
       'влюбиться', 'вода', 'водивший', 'водитель', 'водительский',
       'водить', 'водный', 'водяной', 'возжигание', 'возжигать',
       'возлагаемый', 'возложенный', 'возможный', 'восстановление',
       'восторг', 'впахивать', 'вприскочку', 'вранье', 'врать', 'врач',
       'вскакать', 'вскочить', 'вспахивание', 'вспахивать', 'вспахнутый',
       'вспашка', 'всякий', 'выберу', 'выбирать', 'выгорание',
       'выдернуть', 'выдирать', 'выжегший', 'выкрасить', 'выкрашенный',
       'вылетание'

In [6]:
m = Mystem()

def tag_mystem(m, text):  
    
    processed = m.analyze(text)
    tagged = []
    for w in processed:
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()

            tagged.append(lemma.lower() + '_' + pos)
        except KeyError:
            continue # я здесь пропускаю знаки препинания, но вы можете поступить по-другому
    return tagged

print(tag_mystem(m, 'нарочно банально'))

['нарочно_ADV', 'банальный_A']


In [7]:
unique_words_w_pos = []
for word in unique_words:
    unique_words_w_pos.append(tag_mystem(m, word))

unique_words_w_pos

[['абсолютный_A'],
 ['актер_S'],
 ['активный_A'],
 ['анализ_S'],
 ['анекдот_S'],
 ['анкета_S'],
 ['аргумент_S'],
 ['аромат_S'],
 ['артист_S'],
 ['банально_ADV'],
 ['бегать_V'],
 ['бежать_V'],
 ['беда_S'],
 ['бежать_V'],
 ['безногий_A'],
 ['безнравственный_A'],
 ['беседа_S'],
 ['бесшумный_A'],
 ['бланк_S'],
 ['блестеть_V'],
 ['блестящий_A'],
 ['блистание_S'],
 ['блистательный_A'],
 ['блистательный_A'],
 ['блистать_V'],
 ['блестка_S'],
 ['болезнь_S'],
 ['бур_S'],
 ['бурить_V'],
 ['бурный_A'],
 ['буровой_A'],
 ['бурый_A'],
 ['буря_S'],
 ['бурить_V'],
 ['вариант_S'],
 ['веселие_S'],
 ['вздергивать_V'],
 ['вздирать_V'],
 ['взлет_S'],
 ['взрослый_S'],
 ['влюбляться_V'],
 ['вода_S'],
 ['водить_V'],
 ['водитель_S'],
 ['водительский_A'],
 ['водить_V'],
 ['водный_A'],
 ['водяной_A'],
 ['возжигание_S'],
 ['возжигать_V'],
 ['возлагать_V'],
 ['возлагать_V'],
 ['возможный_A'],
 ['восстановление_S'],
 ['восторг_S'],
 ['впахивать_V'],
 ['вприскочку_ADV'],
 ['вранье_S'],
 ['врать_V'],
 ['врач_S'],
 ['в

# Обработка всех необычных случаев

In [8]:
print(len(unique_words_w_pos))

list_of_pos = []

for el in unique_words_w_pos:
    splitted = el[0].split('_')
    word, pos = splitted[0], splitted[1]
    list_of_pos.append(pos)


print(set(list_of_pos))
    

722
{'SPRO', 'APRO', 'A', 'ADV', 'S', 'V'}


In [9]:
print(len(unique_words_w_pos))

final_list = []
final_list_of_pos = []

for el in unique_words_w_pos:
    splitted = el[0].split('_')
    word, pos = splitted[0], splitted[1]

    if pos == 'APRO': # всякий
        pos = 'ADJ'
    if pos == 'A':
        pos = 'ADJ'
    if pos == 'S':
        pos = 'NOUN'
    if pos == 'SPRO': # любой
        pos = 'ADJ'
    # if pos == 'PR': # слово побег привело в начальную форму и получилось по
    #     word = 'побег'
    #     pos = 'NOUN'
    if pos == 'V':
        pos = 'VERB'
    
    final_list.append(word + '_' + pos)
    final_list_of_pos.append(pos)
    
print(len(final_list))
print(final_list)

722
722
['абсолютный_ADJ', 'актер_NOUN', 'активный_ADJ', 'анализ_NOUN', 'анекдот_NOUN', 'анкета_NOUN', 'аргумент_NOUN', 'аромат_NOUN', 'артист_NOUN', 'банально_ADV', 'бегать_VERB', 'бежать_VERB', 'беда_NOUN', 'бежать_VERB', 'безногий_ADJ', 'безнравственный_ADJ', 'беседа_NOUN', 'бесшумный_ADJ', 'бланк_NOUN', 'блестеть_VERB', 'блестящий_ADJ', 'блистание_NOUN', 'блистательный_ADJ', 'блистательный_ADJ', 'блистать_VERB', 'блестка_NOUN', 'болезнь_NOUN', 'бур_NOUN', 'бурить_VERB', 'бурный_ADJ', 'буровой_ADJ', 'бурый_ADJ', 'буря_NOUN', 'бурить_VERB', 'вариант_NOUN', 'веселие_NOUN', 'вздергивать_VERB', 'вздирать_VERB', 'взлет_NOUN', 'взрослый_NOUN', 'влюбляться_VERB', 'вода_NOUN', 'водить_VERB', 'водитель_NOUN', 'водительский_ADJ', 'водить_VERB', 'водный_ADJ', 'водяной_ADJ', 'возжигание_NOUN', 'возжигать_VERB', 'возлагать_VERB', 'возлагать_VERB', 'возможный_ADJ', 'восстановление_NOUN', 'восторг_NOUN', 'впахивать_VERB', 'вприскочку_ADV', 'вранье_NOUN', 'врать_VERB', 'врач_NOUN', 'вскакивать_VERB

In [1]:
len(final_list_of_pos), len(final_list)

NameError: name 'final_list_of_pos' is not defined

# Замена слов без pos

In [46]:
for i in range(len(final_list)):
    data = data.replace(unique_words.tolist()[i], final_list[i])


data

,Lemma_1,Lemma_2,Type,isCognate
0,вода_NOUN,сыр_NOUN,Неоднокоренные (разные корни),0
1,коса_NOUN,зябь_NOUN,Неоднокоренные (разные корни),0
2,любой_ADJ,водитель_NOUN,Неоднокоренные (разные корни),0
3,нож_NOUN,красить_VERB,Неоднокоренные (разные корни),0
4,красивый_ADJ,переносица_NOUN,Неоднокоренные (разные корни),0
...,...,...,...,...
695,восторг_NOUN,ликование_NOUN,Синонимы,0
696,циничный_ADJ,безнравственный_ADJ,Синонимы,0
697,состоятельность_NOUN,зажиточность_NOUN,Синонимы,0
698,оригинальный_ADJ,самобытный_ADJ,Синонимы,0


In [47]:
data.to_csv('pos_dataset.csv')